# The Hangman Game

* Copyright Trexquant Investment LP. All Rights Reserved.
* Redistribution of this question without written consent from Trexquant is prohibited

## Instruction:
For this coding test, your mission is to write an algorithm that plays the game of Hangman through our API server.

When a user plays Hangman, the server first selects a secret word at random from a list. The server then returns a row of underscores (space separated)—one for each letter in the secret word—and asks the user to guess a letter. If the user guesses a letter that is in the word, the word is redisplayed with all instances of that letter shown in the correct positions, along with any letters correctly guessed on previous turns. If the letter does not appear in the word, the user is charged with an incorrect guess. The user keeps guessing letters until either (1) the user has correctly guessed all the letters in the word
or (2) the user has made six incorrect guesses.

You are required to write a "guess" function that takes current word (with underscores) as input and returns a guess letter. You will use the API codes below to play 1,000 Hangman games. You have the opportunity to practice before you want to start recording your game results.

Your algorithm is permitted to use a training set of approximately 250,000 dictionary words. Your algorithm will be tested on an entirely disjoint set of 250,000 dictionary words. Please note that this means the words that you will ultimately be tested on do NOT appear in the dictionary that you are given. You are not permitted to use any dictionary other than the training dictionary we provided. This requirement will be strictly enforced by code review.

You are provided with a basic, working algorithm. This algorithm will match the provided masked string (e.g. a _ _ l e) to all possible words in the dictionary, tabulate the frequency of letters appearing in these possible words, and then guess the letter with the highest frequency of appearence that has not already been guessed. If there are no remaining words that match then it will default back to the character frequency distribution of the entire dictionary.

This benchmark strategy is successful approximately 18% of the time. Your task is to design an algorithm that significantly outperforms this benchmark.

In [ ]:
import json
import requests
import random
import string
import secrets
import time
import re
import collections
from collections import defaultdict, Counter
import numpy as np
from google.colab import drive
drive.mount('/drive')
try:
    from urllib.parse import parse_qs, urlencode, urlparse
except ImportError:
    from urlparse import parse_qs, urlparse
    from urllib import urlencode

from requests.packages.urllib3.exceptions import InsecureRequestWarning

requests.packages.urllib3.disable_warnings(InsecureRequestWarning)

Drive already mounted at /drive; to attempt to forcibly remount, call drive.mount("/drive", force_remount=True).


In [ ]:
class HangmanAPI(object):
    def __init__(self, access_token=None, session=None, timeout=None):
        self.hangman_url = self.determine_hangman_url()
        self.access_token = access_token
        self.session = session or requests.Session()
        self.timeout = timeout
        self.guessed_letters = []

        # the working envir is colab
        full_dictionary_location = "/drive/My Drive/Colab Notebooks/words_250000_train.txt"
        self.full_dictionary = self.build_dictionary(full_dictionary_location)
        self.full_dictionary_common_letter_sorted = collections.Counter("".join(self.full_dictionary)).most_common()

        self.current_dictionary = []

        self.ngram_n = 7 # construct ngram up to ngram_n

        self.unigram_set = self.unigram(self.full_dictionary) # single letter frequency
        # construtc ngram set
        self.ngram_set = [self.ngram(self.full_dictionary, i) for i in range(2, self.ngram_n+1)]

    def unigram(self, corpus):
        '''
        Construct unigram set

        Parameters:
        ------------
        corpus: list

        Return:
        --------
        unigram_set: Counter
        '''
        # init
        unigram_set = Counter()

        # calc freq
        for word in corpus:
            for char in word:
                unigram_set[char] += 1

        return unigram_set

    def ngram(self, corpus, n):
        '''
        Construct ngram set

        Paramters:
        ------------
        corpus: list
        n: int

        Returns:
        ------------
        ngram_set: list[dict[Counter]]
            n ngram_sub_set will be compact in ngram_set
            e.g. within trigram_set, there are three subset which record all the
                posibility in the blank of '_ab', 'a_b' and 'ab_' respectively
        '''

        # create a list of unique defaultdict(Counter)
        ngram_set = [defaultdict(Counter) for _ in range(n)]

        # using prefix and suffix to hightlight the first and last several letter
        prefix = '*' * (n-1)
        suffix = '#' * (n-1)

        # construct ngram_set
        for word in corpus:
            word = prefix + word + suffix
            combos = zip(*[word[i:len(word)-(n-1)+i] for i in range(n)])
            for combo in combos:
                for i in range(n):
                    if combo[i] == '*' or combo[i] == '#': # we are not guessing * and #
                        pass
                    else:
                        # construct ngram_key, e.g. '_ab', 'a_b' and 'ab_' for trigram
                        ngram_key = ''.join([combo[j] if j!=i else '_' for j in range(n)])
                        # the letter in blank
                        ngram_value = combo[i]
                        # record into ngram_set
                        ngram_set[i][ngram_key][ngram_value] += 1

        return ngram_set

    def prob_ngram(self, idx, char, mask, unguessed, NGRAM_CONSTANTS, ngram_set):
        '''
        Given current index, mask and unguessed letter, calculate the weighted
        prob of char

        Parameters:
        ------------
        idx: int
        char: str
        mask: str
        unguessed: list
        NGRAM_CONSTANTS: list
            list of weight constant for ngram
        ngram_set: list[dict[Counter]]

        Returns:
        ------------
        prob_res: list
        '''
        n = len(ngram_set)

        # searching for ngram based on current idx + idx_delta
        # e.g. for trigram, [[0,1,2],[-1,0,1],[-2,-1,0]]
        #      searching for '_ab', 'a_b' and 'ab_'
        idx_delta_sets = [list(range(0-i,n-i)) for i in range(0,n)]

        # weight constant for ngram
        NGRAM_CONSTANT = NGRAM_CONSTANTS[n-1]
        prob_res = []

        # loop over ngram_set, e.g. for trigram, set for '_ab', 'a_b' and 'ab_'
        for i in range(n):
            # prob init
            prob = 0
            # idx_delta to use, e.g. '_ab' with [0,1,2] being its idx_delta_set
            idx_delta_set = idx_delta_sets[i]

            ngram_key = ''.join([mask[idx+idx_delta] for idx_delta in idx_delta_set])

            # check whether other letters in ngram are not '_'
            idx_delta_set.remove(0)
            is_valid = np.array([mask[idx+idx_delta]!='_' for idx_delta in idx_delta_set]).all()

            # for those ngram with '_' in the first place, check whether index out of range
            if i == 0:
                is_valid = is_valid and (idx < len(mask) - n + 1)

            # calculate weighted probability
            if is_valid:
                if ngram_set[i][ngram_key][char]:
                    # the probility is calculated only among those letters unguessed
                    prob += NGRAM_CONSTANT * ngram_set[i][ngram_key][char] \
                        /sum([ngram_set[i][ngram_key][char] for char in unguessed])

            # prob_res record prob of char over ngram_set
            prob_res.append(prob)

        return prob_res

    @staticmethod
    def determine_hangman_url():
        links = ['https://trexsim.com', 'https://sg.trexsim.com']

        data = {link: 0 for link in links}

        for link in links:

            requests.get(link)

            for i in range(10):
                s = time.time()
                requests.get(link)
                data[link] = time.time() - s

        link = sorted(data.items(), key=lambda x: x[1])[0][0]
        link += '/trexsim/hangman'
        return link

    def guess(self, word): # word input example: "_ p p _ e "
        ngram_size = self.ngram_n

        # skip space in word
        mask = word[::2]

        # add prefix and suffix to mask
        prefix = '*' * (ngram_size-1)
        suffix = '#' * (ngram_size-1)
        mask = prefix + mask + suffix

        # what we can guess from
        unguessed = set(string.ascii_lowercase) - set(self.guessed_letters)

        # weight for ngram_set
        NGRAM_CONSTANTS = [i**2 for i in range(1,ngram_size+1)]

        # see the process
        print(mask)

        # record probability by char
        char_prob = dict()
        for char in unguessed:
            prob = 0
            for idx in range(len(mask)):
                if mask[idx] == '_':
                    # unigram
                    prob += NGRAM_CONSTANTS[1-1]*self.unigram_set[char]/\
                            sum([self.unigram_set[char] for char in unguessed])
                    # ngram
                    for i in range(self.ngram_n-1):
                        prob += np.array(self.prob_ngram(idx, char, mask, \
                           unguessed, NGRAM_CONSTANTS, self.ngram_set[i])).sum()

            char_prob[char] = prob
        # when no letter is correctly guessed, we will use unigram to guess
        if all(c in ['_', '*', '#'] for c in mask):
            guess = sorted(self.unigram_set.items(),
                           key=lambda item: item[1])[-len(self.guessed_letters)-1][0]
        # when at least one letter is correctly guessed, use both unigram and ngram to guess
        else:
            guess = sorted(char_prob.items(), key=lambda item: item[1])[-1][0]

        return guess

    ##########################################################
    # You'll likely not need to modify any of the code below #
    ##########################################################

    def build_dictionary(self, dictionary_file_location):
        text_file = open(dictionary_file_location,"r")
        full_dictionary = text_file.read().splitlines()
        text_file.close()
        return full_dictionary

    def start_game(self, practice=True, verbose=True):
        # reset guessed letters to empty set and current plausible dictionary to the full dictionary
        self.guessed_letters = []
        self.current_dictionary = self.full_dictionary

        response = self.request("/new_game", {"practice":practice})
        if response.get('status')=="approved":
            game_id = response.get('game_id')
            word = response.get('word')
            tries_remains = response.get('tries_remains')
            if verbose:
                print("Successfully start a new game! Game ID: {0}. # of tries remaining: {1}. Word: {2}.".format(game_id, tries_remains, word))
            while tries_remains>0:
                # get guessed letter from user code
                guess_letter = self.guess(word)

                # append guessed letter to guessed letters field in hangman object
                self.guessed_letters.append(guess_letter)
                if verbose:
                    print("Guessing letter: {0}".format(guess_letter))

                try:
                    res = self.request("/guess_letter", {"request":"guess_letter", "game_id":game_id, "letter":guess_letter})
                except HangmanAPIError:
                    print('HangmanAPIError exception caught on request.')
                    continue
                except Exception as e:
                    print('Other exception caught on request.')
                    raise e

                if verbose:
                    print("Sever response: {0}".format(res))
                status = res.get('status')
                tries_remains = res.get('tries_remains')
                if status=="success":
                    if verbose:
                        print("Successfully finished game: {0}".format(game_id))
                    return True
                elif status=="failed":
                    reason = res.get('reason', '# of tries exceeded!')
                    if verbose:
                        print("Failed game: {0}. Because of: {1}".format(game_id, reason))
                    return False
                elif status=="ongoing":
                    word = res.get('word')
        else:
            if verbose:
                print("Failed to start a new game")
        return status=="success"

    def my_status(self):
        return self.request("/my_status", {})

    def request(
            self, path, args=None, post_args=None, method=None):
        if args is None:
            args = dict()
        if post_args is not None:
            method = "POST"

        # Add `access_token` to post_args or args if it has not already been
        # included.
        if self.access_token:
            # If post_args exists, we assume that args either does not exists
            # or it does not need `access_token`.
            if post_args and "access_token" not in post_args:
                post_args["access_token"] = self.access_token
            elif "access_token" not in args:
                args["access_token"] = self.access_token

        time.sleep(0.2)

        num_retry, time_sleep = 50, 2
        for it in range(num_retry):
            try:
                response = self.session.request(
                    method or "GET",
                    self.hangman_url + path,
                    timeout=self.timeout,
                    params=args,
                    data=post_args,
                    verify=False
                )
                break
            except requests.HTTPError as e:
                response = json.loads(e.read())
                raise HangmanAPIError(response)
            except requests.exceptions.SSLError as e:
                if it + 1 == num_retry:
                    raise
                time.sleep(time_sleep)

        headers = response.headers
        if 'json' in headers['content-type']:
            result = response.json()
        elif "access_token" in parse_qs(response.text):
            query_str = parse_qs(response.text)
            if "access_token" in query_str:
                result = {"access_token": query_str["access_token"][0]}
                if "expires" in query_str:
                    result["expires"] = query_str["expires"][0]
            else:
                raise HangmanAPIError(response.json())
        else:
            raise HangmanAPIError('Maintype was not text, or querystring')

        if result and isinstance(result, dict) and result.get("error"):
            raise HangmanAPIError(result)
        return result

class HangmanAPIError(Exception):
    def __init__(self, result):
        self.result = result
        self.code = None
        try:
            self.type = result["error_code"]
        except (KeyError, TypeError):
            self.type = ""

        try:
            self.message = result["error_description"]
        except (KeyError, TypeError):
            try:
                self.message = result["error"]["message"]
                self.code = result["error"].get("code")
                if not self.type:
                    self.type = result["error"].get("type", "")
            except (KeyError, TypeError):
                try:
                    self.message = result["error_msg"]
                except (KeyError, TypeError):
                    self.message = result

        Exception.__init__(self, self.message)

# API Usage Examples

## To start a new game:
1. Make sure you have implemented your own "guess" method.
2. Use the access_token that we sent you to create your HangmanAPI object.
3. Start a game by calling "start_game" method.
4. If you wish to test your function without being recorded, set "practice" parameter to 1.
5. Note: You have a rate limit of 20 new games per minute. DO NOT start more than 20 new games within one minute.

In [ ]:
api = HangmanAPI(access_token="7c3e20f806d526000515bab51646fd", timeout=2000)


## Playing practice games:
You can use the command below to play up to 100,000 practice games.

In [ ]:
for _ in range(1):
    api.start_game(practice=1,verbose=True)
    [total_practice_runs,total_recorded_runs,total_recorded_successes,total_practice_successes] = api.my_status() # Get my game stats: (# of tries, # of wins)
    practice_success_rate = total_practice_successes / total_practice_runs
    print('run %d practice games out of an allotted 100,000. practice success rate so far = %.3f' % (total_practice_runs, practice_success_rate))


Successfully start a new game! Game ID: bc1d94ed0960. # of tries remaining: 6. Word: _ _ _ _ _ _ _ _ _ .
******_________######
Guessing letter: e
Sever response: {'game_id': 'bc1d94ed0960', 'status': 'ongoing', 'tries_remains': 6, 'word': '_ e _ _ _ _ e _ _ '}
******_e____e__######
Guessing letter: s
Sever response: {'game_id': 'bc1d94ed0960', 'status': 'ongoing', 'tries_remains': 6, 'word': 's e _ _ s _ e _ s '}
******se__s_e_s######
Guessing letter: r
Sever response: {'game_id': 'bc1d94ed0960', 'status': 'ongoing', 'tries_remains': 6, 'word': 's e _ _ s _ e r s '}
******se__s_ers######
Guessing letter: t
Sever response: {'game_id': 'bc1d94ed0960', 'status': 'ongoing', 'tries_remains': 6, 'word': 's e _ _ s t e r s '}
******se__sters######
Guessing letter: a
Sever response: {'game_id': 'bc1d94ed0960', 'status': 'ongoing', 'tries_remains': 6, 'word': 's e a _ s t e r s '}
******sea_sters######
Guessing letter: m
Sever response: {'game_id': 'bc1d94ed0960', 'status': 'success', 'tries_re

In [ ]:
api.my_status()

[3105, 0, 0, 1941]

## Playing recorded games:
Please finalize your code prior to running the cell below. Once this code executes once successfully your submission will be finalized. Our system will not allow you to rerun any additional games.

Please note that it is expected that after you successfully run this block of code that subsequent runs will result in the error message "Your account has been deactivated".

Once you've run this section of the code your submission is complete. Please send us your source code via email.

In [ ]:
for i in range(1000):
    print('Playing ', i, ' th game')
    # Uncomment the following line to execute your final runs. Do not do this until you are satisfied with your submission
    api.start_game(practice=0,verbose=False)

    # DO NOT REMOVE as otherwise the server may lock you out for too high frequency of requests
    time.sleep(0.5)

Streaming output truncated to the last 5000 lines.
******_________######
******_____e___######
******_____e___######
******_____e___######
******__n__e___######
******__n__e___######
******t_n__et__######
******t_n__et__######
******tono_et__######
******tonomet__######
******tonometr_######
Playing  566  th game
******_____######
******_____######
******___i_######
******___i_######
******___i_######
******a_ai_######
******a_ai_######
******alai_######
******alai_######
Playing  567  th game
******_______######
******_____e_######
******_____ed######
******_____ed######
******____sed######
******____sed######
******_oo_sed######
******_oo_sed######
******_oo_sed######
******woo_sed######
******woo_sed######
Playing  568  th game
******__________######
******_e_e_____e######
******_e_e_____e######
******_e_e____re######
******_e_e____re######
******_e_e___ore######
******_e_e___ore######
******_e_e___ore######
******_e_e___ore######
******se_e_s_ore######
******se_e_score######
Playin

HangmanAPIError: ignored

In [ ]:
api.my_status()

[3106, 1000, 608, 1942]

## To check your game statistics
1. Simply use "my_status" method.
2. Returns your total number of games, and number of wins.

In [ ]:
[total_practice_runs,total_recorded_runs,total_recorded_successes,total_practice_successes] = api.my_status() # Get my game stats: (# of tries, # of wins)
success_rate = total_recorded_successes/total_recorded_runs
print('overall success rate = %.3f' % success_rate)

overall success rate = 0.608
